In [ ]:
import json
import pickle
import pandas as pd
import os

import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.data import Field, BucketIterator
import torch.nn.functional as F
import spacy
import numpy as np

import random
import math
import time

from torchtext.data import TabularDataset

In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
!wget "https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json"

!wget "https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json"

--2020-12-27 08:43:16--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.109.153, 185.199.110.153, 185.199.108.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘train-v2.0.json.2’

train-v2.0.json.2   100%[===================>]  40.17M   251MB/s    in 0.2s    

2020-12-27 08:43:16 (251 MB/s) - ‘train-v2.0.json.2’ saved [42123633/42123633]

--2020-12-27 08:43:16--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.109.153, 185.199.110.153, 185.199.108.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘dev-v2.0.json.2’

dev-v2.0.json.2

In [ ]:
def parse_squad(dataset):
    """
    Parses SQUAD database into more readable format. In this case I only care
    about question/answers pairs in order to make a seq2seq model that would
    generate questions out of a paragraph.
    
    Inputs:
        dataset: squad dataset in json format
    Returns:
        squad_json: parsed squad dataset in json format
    """
    total_topics = 0
    total_questions = 0
    squad_json = []
    
    # Iterate through every topic in the dataset
    for topic in dataset:
        total_topics += 1
        # Iterate through every text passage in the topic
        for passage in topic['paragraphs']:     
            # Iterate through every question/answer pairs in the passage
            for qas in passage['qas']:
                total_questions += 1
                text_question_pair = {}
                # Append the title
                text_question_pair['topic'] = topic['title']   
                # Append the text paragraph
                text_question_pair['paragraph'] = passage['context']
                # Append the question
                text_question_pair['question'] = qas['question']
                # Iterate through available answers
                answers = []
                for answer in qas['answers']:
                    answers.append(answer['text'])
                # And append them all together
                text_question_pair['answers'] = answers 
                
                # Append found dictionary to the full parsed dataset array
                squad_json.append(text_question_pair)
    
    print('Found ' + str(total_topics) + ' topics in total.')
    print('Found ' + str(total_questions) + ' questions in total.')
    return squad_json

In [ ]:
squad_train_filepath = './train-v2.0.json'
save_path = './parsed_train_data.json'

# Load squad train dataset
train_json = json.load(open(squad_train_filepath, 'r'))
train_dataset = train_json['data']

parsed_train_squad = parse_squad(train_dataset)
json.dump(parsed_train_squad, open(save_path, 'w'))

Found 442 topics in total.
Found 130319 questions in total.


In [ ]:
 
squad_dev_filepath = './dev-v2.0.json'
save_path = './parsed_dev_data.json'

# Load squad dev dataset
dev_json = json.load(open(squad_dev_filepath, 'r'))
dev_dataset = dev_json['data']

parsed_dev_squad = parse_squad(dev_dataset)
json.dump(parsed_dev_squad, open(save_path, 'w'))

Found 35 topics in total.
Found 11873 questions in total.


In [ ]:

train_filepath = './parsed_train_data.json'
dev_filepath = './parsed_dev_data.json'

train_set = json.load(open(train_filepath, 'r'))
dev_set = json.load(open(dev_filepath, 'r'))

train_paragraphs = []
train_questions = []
train_answers = []
for section in train_set:
  if len(section['answers']) != 0:
    train_paragraphs.append(section['paragraph'])
    train_questions.append(section['question'])
    train_answers.append(section['answers'][0])
    
dev_paragraphs = []
dev_questions = []
dev_answers = []
for section in dev_set:
  if len(section['answers']) != 0:
    dev_paragraphs.append(section['paragraph'])
    dev_questions.append(section['question'])
    dev_answers.append(section['answers'][0])
assert len(train_paragraphs) == len(train_questions)
assert len(dev_paragraphs) == len(dev_questions)

In [ ]:
train_dict = {'Questions':train_questions,'Answers':train_answers}
train_data = pd.DataFrame(train_dict)
train_data.head()

,Questions,Answers
0,When did Beyonce start becoming popular?,in the late 1990s
1,What areas did Beyonce compete in when she was...,singing and dancing
2,When did Beyonce leave Destiny's Child and bec...,2003
3,In what city and state did Beyonce grow up?,"Houston, Texas"
4,In which decade did Beyonce become famous?,late 1990s


In [ ]:
if not os.path.exists('preprocessed_train.csv'):
    train_data.to_csv('preprocessed_train.csv', index = False)

In [ ]:
valid_dict = {'Questions':dev_questions,'Answers':dev_answers}
valid_data = pd.DataFrame(valid_dict)
valid_data.head()

,Questions,Answers
0,In what country is Normandy located?,France
1,When were the Normans in Normandy?,10th and 11th centuries
2,From which countries did the Norse originate?,"Denmark, Iceland and Norway"
3,Who was the Norse leader?,Rollo
4,What century did the Normans first gain their ...,10th century


In [ ]:
if not os.path.exists('preprocessed_valid.csv'):
    valid_data.to_csv('preprocessed_valid.csv', index = False)

In [ ]:
spacy_en = spacy.load('en')

In [ ]:
def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [ ]:
QSTN = Field(tokenize=tokenize_en, 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=True)

ANS = Field(tokenize = tokenize_en, 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=True)

In [ ]:
trainset = TabularDataset(path='preprocessed_train.csv', format='csv', skip_header=True,
                            fields=[('Question', QSTN), ('Answer', ANS)])

In [ ]:
test_data = TabularDataset(path='preprocessed_valid.csv', format='csv', skip_header=True,
                            fields=[('Question', QSTN), ('Answer', ANS)])

In [ ]:
train_data, valid_data = trainset.split(split_ratio = 0.7, random_state=random.getstate())                                

In [ ]:
vars(train_data.examples[0])

{'Answer': ['slowed',
  'down',
  'and',
  'flattened',
  'along',
  'the',
  'spaceship',
  "'s",
  'direction',
  'of',
  'travel'],
 'Question': ['how',
  'does',
  'the',
  'crew',
  'on',
  'board',
  'the',
  'spaceship',
  'perceive',
  'the',
  'stationary',
  'observer',
  '?']}

In [ ]:
vars(valid_data.examples[0])

{'Answer': ['wholesale', 'and', 'retail'],
 'Question': ['what',
  "'s",
  'the',
  'third',
  'largest',
  'sector',
  'providing',
  'jobs',
  'in',
  'southampton',
  '?']}

In [ ]:
QSTN.build_vocab(train_data, min_freq = 2)
ANS.build_vocab(train_data, min_freq = 2)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device, sort_key=lambda x: len(x.Question), # the BucketIterator needs to be told what function it should use to group the data.
 sort_within_batch=False,)


In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded)
                
        #outputs = [src len, batch size, hid dim * num directions]
        #hidden = [n layers * num directions, batch size, hid dim]
        
        #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        #outputs are always from the last layer
        
        #hidden [-2, :, : ] is the last of the forwards RNN 
        #hidden [-1, :, : ] is the last of the backwards RNN
        
        #initial decoder hidden is final hidden state of the forwards and backwards 
        #  encoder RNNs fed through a linear layer
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        
        #outputs = [src len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]
        
        return outputs, hidden

In [ ]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)
        
    def forward(self, hidden, encoder_outputs):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat decoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #hidden = [batch size, src len, dec hid dim]
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        
        #energy = [batch size, src len, dec hid dim]

        attention = self.v(energy).squeeze(2)
        
        #attention= [batch size, src len]
        
        return F.softmax(attention, dim=1)

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
        
        a = self.attention(hidden, encoder_outputs)
                
        #a = [batch size, src len]
        
        a = a.unsqueeze(1)
        
        #a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        weighted = torch.bmm(a, encoder_outputs)
        
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        
        #weighted = [1, batch size, enc hid dim * 2]
        
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
            
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        #output = [seq len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #seq len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden.squeeze(0)

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)
                
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and all encoder hidden states
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

In [ ]:
INPUT_DIM = len(QSTN.vocab)
OUTPUT_DIM = len(ANS.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(17687, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(15004, 256)
    (rnn): GRU(1280, 512)
    (fc_out): Linear(in_features=1792, out_features=15004, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 41,704,348 trainable parameters


In [ ]:
optimizer = optim.Adam(model.parameters())

In [ ]:
TRG_PAD_IDX = ANS.vocab.stoi[ANS.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.Question
        trg = batch.Answer
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.Question
            trg = batch.Answer

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 4
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 4m 28s
	Train Loss: 6.190 | Train PPL: 487.864
	 Val. Loss: 5.503 |  Val. PPL: 245.490
Epoch: 02 | Time: 4m 28s
	Train Loss: 5.509 | Train PPL: 246.855
	 Val. Loss: 5.272 |  Val. PPL: 194.798
Epoch: 03 | Time: 4m 28s
	Train Loss: 5.109 | Train PPL: 165.451
	 Val. Loss: 5.186 |  Val. PPL: 178.745
Epoch: 04 | Time: 4m 30s
	Train Loss: 4.712 | Train PPL: 111.274
	 Val. Loss: 5.194 |  Val. PPL: 180.181


In [ ]:
model.load_state_dict(torch.load('tut3-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 5.316 | Test PPL: 203.469 |
